In [1]:
%%capture
pip install numpy pandas

In [2]:
import numpy as np
import pandas as pd

### **Cohorts and academies_alias**

In [3]:
cohorts = pd.read_csv('../sources/cohorts (2).csv')

In [4]:
cohorts

,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,updated_at,...,timezone,private,never_ends,schedule_id,syllabus_version_id,online_meeting_url,remote_available,current_module,history_log,is_hidden_on_prework
0,210,pre-work-latam,Pre-work-latam,2021-07-07 02:04:18.906000+00:00,NaN,0,STARTED,en,2021-07-07 02:04:57.437696+00:00,2023-02-17 05:47:05.271083+00:00,...,America/Bogota,False,True,83.0,101.0,https://whereby.com/pbullor,True,NaN,NaN,False
1,374,sample-2,Sample-2,2022-05-25 19:20:28.798000+00:00,2023-03-16 19:20:00+00:00,1,STARTED,EN,2022-05-25 19:21:29.887390+00:00,2022-08-30 22:23:16.481231+00:00,...,America/Costa_Rica,False,False,72.0,104.0,NaN,False,1.0,NaN,NaN
2,300,orlando-pt-5,Orlando PT 5,2022-04-05 02:53:00+00:00,2022-07-30 02:53:00+00:00,0,INACTIVE,en,2022-01-25 03:54:14.088816+00:00,2022-01-25 03:54:14.088828+00:00,...,NaN,False,False,1.0,27.0,NaN,True,NaN,NaN,NaN
3,200,caracas-pt-xxvi,Caracas PT XXVI,2021-05-25 00:00:00+00:00,2021-09-04 00:00:00+00:00,48,ENDED,es,2021-04-12 23:00:58.115104+00:00,2021-11-03 15:09:23.017555+00:00,...,Africa/Abidjan,False,False,NaN,19.0,NaN,True,NaN,NaN,NaN
4,528,land-a-job-in-tech-europe,Land a Job in Tech Europe,2023-02-22 10:35:52.072000+00:00,NaN,0,STARTED,en,2023-02-22 10:38:26.079310+00:00,2023-02-22 10:38:36.567149+00:00,...,Europe/Madrid,False,True,138.0,181.0,https://meet.google.com,True,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,465,madrid-pt-33,Madrid PT 33,2022-11-22 13:26:00+00:00,2023-03-30 13:26:00+00:00,33,STARTED,es,2022-09-28 13:28:54.662204+00:00,2023-02-25 11:37:31.622555+00:00,...,Europe/Madrid,False,False,25.0,61.0,https://meet.google.com/zxp-grsc-kun,True,26.0,"{""1"": {""updated_at"": ""2022-12-01 17:17:01.3726...",NaN
391,487,latam-ft-uy-2,latam-ft-uy-2,2022-12-12 14:30:00+00:00,2023-02-28 14:30:00+00:00,45,STARTED,es,2022-11-14 14:31:10.551097+00:00,2023-02-21 21:04:52.894162+00:00,...,America/Montevideo,False,False,122.0,11.0,https://meet.google.com/eim-uzvt-dhk,True,37.0,"{""1"": {""updated_at"": ""2022-12-12 20:16:33.0839...",NaN
392,401,miami-48,Miami 48,2023-01-30 15:54:00+00:00,2023-06-02 14:54:00+00:00,9,STARTED,en,2022-08-11 14:54:38.932243+00:00,2023-02-22 23:23:46.708907+00:00,...,America/New_York,False,False,1.0,100.0,https://bluejeans.com/554143921/5622,True,8.0,"{""1"": {""updated_at"": ""2023-01-31 01:57:28.5420...",False
393,508,santiago-ft-19,Santiago FT-19,2023-02-06 13:25:00+00:00,2023-04-14 14:25:00+00:00,15,STARTED,es,2022-12-12 13:28:39.988427+00:00,2023-02-24 15:51:11.382458+00:00,...,America/Santiago,False,False,21.0,11.0,https://bluejeans.com/524605273/2228,True,19.0,"{""1"": {""updated_at"": ""2023-02-07 12:36:27.4555...",NaN


In [5]:
cohorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    395 non-null    int64  
 1   slug                  395 non-null    object 
 2   name                  395 non-null    object 
 3   kickoff_date          395 non-null    object 
 4   ending_date           355 non-null    object 
 5   current_day           395 non-null    int64  
 6   stage                 395 non-null    object 
 7   language              395 non-null    object 
 8   created_at            395 non-null    object 
 9   updated_at            395 non-null    object 
 10  academy_id            395 non-null    int64  
 11  timezone              205 non-null    object 
 12  private               395 non-null    bool   
 13  never_ends            395 non-null    bool   
 14  schedule_id           195 non-null    float64
 15  syllabus_version_id   3

In [6]:
academies_alias = pd.read_csv('../sources/academies_alias.csv')

In [7]:
academies_alias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   slug        29 non-null     object
 1   academy_id  29 non-null     int64 
 2   name        29 non-null     object
 3   status      29 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.0+ KB


In [8]:
academies_alias

,slug,academy_id,name,status
0,caracas-venezuela,2,4Geeks Academy Caracas,ACTIVE
1,maracaibo,2,4Geeks Academy Caracas,ACTIVE
2,toronto-canada,4,4Geeks Academy Miami,ACTIVE
3,downtown-miami,4,4Geeks Academy Miami,ACTIVE
4,orlando,11,4Geeks Academy Orlando,ACTIVE
5,milan-italy,10,4Geeks Academy Europe,ACTIVE
6,rome-italy,10,4Geeks Academy Europe,ACTIVE
7,berlin-germany,10,4Geeks Academy Europe,ACTIVE
8,hamburg-germany,10,4Geeks Academy Europe,ACTIVE
9,dublin-ireland,10,4Geeks Academy Europe,ACTIVE


In [9]:
academies_alias = academies_alias.rename(columns={'name':'campus'}).drop(columns=['slug', 'status']).drop_duplicates()

In [10]:
academies_alias

,academy_id,campus
0,2,4Geeks Academy Caracas
2,4,4Geeks Academy Miami
4,11,4Geeks Academy Orlando
5,10,4Geeks Academy Europe
13,6,4Geeks Academy Madrid
17,5,4Geeks Academy Santiago
18,7,4Geeks Academy Latam


In [11]:
cohorts = cohorts.merge(academies_alias, how='left', on='academy_id')

In [12]:
cohorts

,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,updated_at,...,private,never_ends,schedule_id,syllabus_version_id,online_meeting_url,remote_available,current_module,history_log,is_hidden_on_prework,campus
0,210,pre-work-latam,Pre-work-latam,2021-07-07 02:04:18.906000+00:00,NaN,0,STARTED,en,2021-07-07 02:04:57.437696+00:00,2023-02-17 05:47:05.271083+00:00,...,False,True,83.0,101.0,https://whereby.com/pbullor,True,NaN,NaN,False,4Geeks Academy Latam
1,374,sample-2,Sample-2,2022-05-25 19:20:28.798000+00:00,2023-03-16 19:20:00+00:00,1,STARTED,EN,2022-05-25 19:21:29.887390+00:00,2022-08-30 22:23:16.481231+00:00,...,False,False,72.0,104.0,NaN,False,1.0,NaN,NaN,NaN
2,300,orlando-pt-5,Orlando PT 5,2022-04-05 02:53:00+00:00,2022-07-30 02:53:00+00:00,0,INACTIVE,en,2022-01-25 03:54:14.088816+00:00,2022-01-25 03:54:14.088828+00:00,...,False,False,1.0,27.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Orlando
3,200,caracas-pt-xxvi,Caracas PT XXVI,2021-05-25 00:00:00+00:00,2021-09-04 00:00:00+00:00,48,ENDED,es,2021-04-12 23:00:58.115104+00:00,2021-11-03 15:09:23.017555+00:00,...,False,False,NaN,19.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Caracas
4,528,land-a-job-in-tech-europe,Land a Job in Tech Europe,2023-02-22 10:35:52.072000+00:00,NaN,0,STARTED,en,2023-02-22 10:38:26.079310+00:00,2023-02-22 10:38:36.567149+00:00,...,False,True,138.0,181.0,https://meet.google.com,True,NaN,NaN,True,4Geeks Academy Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,465,madrid-pt-33,Madrid PT 33,2022-11-22 13:26:00+00:00,2023-03-30 13:26:00+00:00,33,STARTED,es,2022-09-28 13:28:54.662204+00:00,2023-02-25 11:37:31.622555+00:00,...,False,False,25.0,61.0,https://meet.google.com/zxp-grsc-kun,True,26.0,"{""1"": {""updated_at"": ""2022-12-01 17:17:01.3726...",NaN,4Geeks Academy Madrid
391,487,latam-ft-uy-2,latam-ft-uy-2,2022-12-12 14:30:00+00:00,2023-02-28 14:30:00+00:00,45,STARTED,es,2022-11-14 14:31:10.551097+00:00,2023-02-21 21:04:52.894162+00:00,...,False,False,122.0,11.0,https://meet.google.com/eim-uzvt-dhk,True,37.0,"{""1"": {""updated_at"": ""2022-12-12 20:16:33.0839...",NaN,4Geeks Academy Latam
392,401,miami-48,Miami 48,2023-01-30 15:54:00+00:00,2023-06-02 14:54:00+00:00,9,STARTED,en,2022-08-11 14:54:38.932243+00:00,2023-02-22 23:23:46.708907+00:00,...,False,False,1.0,100.0,https://bluejeans.com/554143921/5622,True,8.0,"{""1"": {""updated_at"": ""2023-01-31 01:57:28.5420...",False,4Geeks Academy Miami
393,508,santiago-ft-19,Santiago FT-19,2023-02-06 13:25:00+00:00,2023-04-14 14:25:00+00:00,15,STARTED,es,2022-12-12 13:28:39.988427+00:00,2023-02-24 15:51:11.382458+00:00,...,False,False,21.0,11.0,https://bluejeans.com/524605273/2228,True,19.0,"{""1"": {""updated_at"": ""2023-02-07 12:36:27.4555...",NaN,4Geeks Academy Santiago


In [13]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [14]:
cohorts['kickoff_date'] = cohorts['kickoff_date'].apply(correct_date)
cohorts['ending_date'] = cohorts['ending_date'].apply(correct_date)
cohorts['created_at'] = cohorts['created_at'].apply(correct_date)
cohorts['updated_at'] = cohorts['updated_at'].apply(correct_date)

In [15]:
cohorts['datetime_kickoff'] = pd.to_datetime(cohorts['kickoff_date'])

In [16]:
current_time = pd.Timestamp.now()

In [17]:
upcoming_cohorts = cohorts[cohorts['datetime_kickoff']>current_time]

In [18]:
upcoming_cohorts = upcoming_cohorts[upcoming_cohorts['never_ends']==False]

In [19]:
upcoming_cohorts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 126 to 384
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    33 non-null     int64         
 1   slug                  33 non-null     object        
 2   name                  33 non-null     object        
 3   kickoff_date          33 non-null     object        
 4   ending_date           33 non-null     object        
 5   current_day           33 non-null     int64         
 6   stage                 33 non-null     object        
 7   language              33 non-null     object        
 8   created_at            33 non-null     object        
 9   updated_at            33 non-null     object        
 10  academy_id            33 non-null     int64         
 11  timezone              33 non-null     object        
 12  private               33 non-null     bool          
 13  never_ends         

In [20]:
upcoming_cohorts['stage'].value_counts()

INACTIVE    20
DELETED     11
STARTED      1
PREWORK      1
Name: stage, dtype: int64

In [21]:
upcoming_cohorts = upcoming_cohorts[(upcoming_cohorts['stage']!='DELETED')]

In [22]:
upcoming_cohorts.columns

Index(['id', 'slug', 'name', 'kickoff_date', 'ending_date', 'current_day',
       'stage', 'language', 'created_at', 'updated_at', 'academy_id',
       'timezone', 'private', 'never_ends', 'schedule_id',
       'syllabus_version_id', 'online_meeting_url', 'remote_available',
       'current_module', 'history_log', 'is_hidden_on_prework', 'campus',
       'datetime_kickoff'],
      dtype='object')

In [23]:
upcoming_cohorts

,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,updated_at,...,never_ends,schedule_id,syllabus_version_id,online_meeting_url,remote_available,current_module,history_log,is_hidden_on_prework,campus,datetime_kickoff
126,501,spain-fs-pt-enero,Spain FS PT Enero,2023-06-06 14:16:00,2023-10-10 15:16:00,0,INACTIVE,es,2022-12-09 15:17:36.465700,2022-12-24 10:42:58.373921,...,False,128.0,182.0,NaN,False,NaN,NaN,NaN,4Geeks Academy Madrid,2023-06-06 14:16:00
216,503,spain-fs-pt-marzo,Spain FS PT Marzo,2023-03-07 15:23:00,2023-07-18 15:23:00,0,INACTIVE,es,2022-12-09 15:24:20.195068,2022-12-09 15:24:20.195078,...,False,128.0,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Madrid,2023-03-07 15:23:00
217,380,fake-ml-cohort,Fake ML Cohort,2023-04-20 19:04:00,2023-12-19 20:04:00,0,STARTED,en,2022-06-15 19:04:29.787991,2023-02-02 13:40:50.675166,...,False,77.0,136.0,NaN,False,NaN,NaN,True,NaN,2023-04-20 19:04:00
222,473,latam-pt-20,latam-pt-20,2023-06-19 18:16:00,2023-10-20 18:16:00,0,INACTIVE,es,2022-10-18 18:17:28.070083,2023-01-16 22:41:35.109890,...,False,134.0,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Latam,2023-06-19 18:16:00
223,470,latam-pt-17,latam-pt-17,2023-03-13 18:12:00,2023-07-17 18:12:00,0,INACTIVE,es,2022-10-18 18:14:09.717460,2023-01-16 22:42:17.213203,...,False,134.0,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Latam,2023-03-13 18:12:00
254,403,miami-50,Miami 50,2023-03-20 14:55:00,2023-07-21 14:55:00,0,PREWORK,en,2022-08-11 14:55:57.057573,2023-02-22 19:10:04.686012,...,False,1.0,61.0,https://google.com,True,NaN,NaN,False,4Geeks Academy Miami,2023-03-20 14:55:00
255,404,miami-51,Miami 51,2023-04-10 14:55:00,2023-08-11 14:55:00,0,INACTIVE,en,2022-08-11 14:56:35.541142,2023-02-13 14:19:28.670074,...,False,1.0,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-04-10 14:55:00
256,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,False,13.0,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-01 14:56:00
257,406,miami-53,Miami 53,2023-05-22 14:57:00,2023-09-22 14:57:00,0,INACTIVE,en,2022-08-11 14:58:49.366592,2023-02-13 14:20:45.477173,...,False,1.0,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-22 14:57:00
264,504,spain-fs-pt-abril,Spain FS PT Abril,2023-04-10 15:24:00,2023-08-14 15:24:00,0,INACTIVE,es,2022-12-09 15:26:39.048445,2022-12-09 15:26:39.048456,...,False,127.0,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Madrid,2023-04-10 15:24:00


### **Unconfirmed**

In [24]:
form_entry = pd.read_csv('../output/form_entry_processed.csv', low_memory=False)

In [25]:
form_entry.columns

Index(['id', 'first_name', 'last_name', 'email', 'phone', 'course',
       'client_comments', 'language', 'utm_url', 'utm_medium', 'utm_campaign',
       'street_address', 'country', 'city', 'latitude', 'longitude', 'state',
       'zip_code', 'storage_status', 'created_at', 'updated_at', 'contact_id',
       'lead_type', 'location', 'referral_key', 'automations', 'tags', 'gclid',
       'browser_lang', 'fb_ad_id', 'fb_adgroup_id', 'fb_form_id',
       'fb_leadgen_id', 'fb_page_id', 'utm_source', 'academy_id',
       'deal_status', 'sentiment', 'ac_contact_id', 'ac_deal_id', 'user_id',
       'won_at', 'ac_expected_cohort', 'current_download',
       'lead_generation_app_id', 'utm_content', 'storage_status_text',
       'custom_fields', 'sex', 'utm_placement', 'utm_plan', 'utm_term',
       'ac_deal_owner_full_name', 'ac_deal_owner_id',
       'ac_expected_cohort_date', 'ac_deal_amount', 'ac_deal_currency_code',
       'datetime_form_entry_creation', '_merge', 'to_drop', 'fullname',
  

In [26]:
form_entry['datetime_update'] = pd.to_datetime(form_entry['updated_at'])

In [27]:
dict_assigned = form_entry.groupby('ac_expected_cohort')['email'].apply(list).to_dict()

In [28]:
dict_assigned

{'caracas-pt-42': ['josemfcheo@gmail.com'],
 'cohort-34': ['justjesi2010@gmail.com'],
 'fake-ml-cohort': ['a.alejo@gmail.com'],
 'full-stack-11': ['calexandergarcia@gmail.com'],
 'full-stack-21': ['wambuangui0746@gmail.com'],
 'miami-50': ['test@4geeksacademy.com'],
 'miami-52': ['stallonedylan954@gmail.com', 'a.a.lejo@gmail.com'],
 'spain-fs-pt-mayo': ['mazda3mps265@hotmail.com']}

### **Confirmed**

In [29]:
cohort_users = pd.read_csv('../sources/modified_cohort_users.csv')

In [30]:
cohort_users.columns

Index(['id', 'role', 'finantial_status', 'educational_status', 'created_at',
       'updated_at', 'cohort_id', 'user_id', 'watching', 'history_log'],
      dtype='object')

In [31]:
cohorts_slugs = cohorts[['id', 'slug']].drop_duplicates()

In [32]:
cohort_users = cohort_users.merge(cohorts_slugs, left_on='cohort_id', right_on='id', how='left')
cohort_users = cohort_users.rename(columns={'id_x':'id'})
cohort_users = cohort_users.drop(columns=['id_y'])

In [33]:
auth = pd.read_csv('../sources/auth_user.csv')

In [34]:
cohort_users_auth = cohort_users.merge(auth, left_on='user_id', right_on='id', how='left')

In [35]:
cohort_users_auth.sample(3)

,id_x,role,finantial_status,educational_status,created_at,updated_at,cohort_id,user_id,watching,history_log,slug,id_y,first_name,last_name,email,date_joined
6,9,STUDENT,NaN,GRADUATED,2020-08-19 23:06:31.584120+00:00,2020-08-19 23:06:31.584140+00:00,473,18,False,{},latam-pt-20,18,Michael,Venereo,hexen6@gmail.com,2020-08-19 23:06:31.578840+00:00
2,4,student,LATE,SUSPENDED,2020-08-19 23:06:31.538438+00:00,2020-08-19 23:06:31.538457+00:00,406,14,False,{},miami-53,14,Fake,Studenth,a+fakestudent8@4geeks.us,2020-08-19 23:06:31.513344+00:00
1,3,STUDENT,LATE,SUSPENDED,2020-08-19 23:06:31.532619+00:00,2020-08-19 23:06:31.532644+00:00,405,14,False,{},miami-52,14,Fake,Studenth,a+fakestudent8@4geeks.us,2020-08-19 23:06:31.513344+00:00


In [36]:
cohort_users_auth.columns

Index(['id_x', 'role', 'finantial_status', 'educational_status', 'created_at',
       'updated_at', 'cohort_id', 'user_id', 'watching', 'history_log', 'slug',
       'id_y', 'first_name', 'last_name', 'email', 'date_joined'],
      dtype='object')

In [37]:
cohort_users_auth['role'].value_counts()

STUDENT    6
student    1
OTRO       1
Name: role, dtype: int64

In [38]:
cohort_users_auth = cohort_users_auth[cohort_users_auth['role'].str.lower()=='student']

In [39]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [40]:
cohort_users_auth['updated_at'] = cohort_users_auth['updated_at'].apply(correct_date)

In [41]:
cohort_users_auth['datetime_update'] = pd.to_datetime(cohort_users_auth['updated_at'])

In [42]:
dict_confirmed = cohort_users_auth.groupby('slug')['email'].apply(list).to_dict()

**Create final dataframe**

In [43]:
list_slugs = list(dict_assigned.keys()) + list(dict_confirmed.keys())

In [44]:
list_slugs = list(set(list_slugs))

In [45]:
list_slugs

['spain-fs-pt-marzo',
 'spain-fs-pt-enero',
 'miami-52',
 'mdc-iii',
 'miami-51',
 'spain-fs-pt-mayo',
 'full-stack-11',
 'full-stack-21',
 'cohort-34',
 'latam-pt-20',
 'miami-53',
 'fake-ml-cohort',
 'miami-50',
 'caracas-pt-42']

In [46]:
dict_slug_email = {}

for slug in list_slugs:
    
    dict_slug_email[slug] = []

    if slug in list(dict_assigned.keys()):
        dict_slug_email[slug].extend(dict_assigned[slug])
    
    if slug in list(dict_confirmed.keys()):
        dict_slug_email[slug].extend(dict_confirmed[slug])

In [47]:
dict_slug_email

{'spain-fs-pt-marzo': ['edgarbrignoni@icloud.com'],
 'spain-fs-pt-enero': ['alejandro.leon1223@gmail.com'],
 'miami-52': ['stallonedylan954@gmail.com',
  'a.a.lejo@gmail.com',
  'a+fakestudent8@4geeks.us'],
 'mdc-iii': ['imjosearce@gmail.com'],
 'miami-51': ['a+fakestudent8@4geeks.us'],
 'spain-fs-pt-mayo': ['mazda3mps265@hotmail.com'],
 'full-stack-11': ['calexandergarcia@gmail.com'],
 'full-stack-21': ['wambuangui0746@gmail.com'],
 'cohort-34': ['justjesi2010@gmail.com'],
 'latam-pt-20': ['hexen6@gmail.com'],
 'miami-53': ['a+fakestudent8@4geeks.us'],
 'fake-ml-cohort': ['a.alejo@gmail.com'],
 'miami-50': ['test@4geeksacademy.com'],
 'caracas-pt-42': ['josemfcheo@gmail.com']}

In [48]:
list_slug_email = []
for key, value in dict_slug_email.items():
    for item in value:
        list_slug_email.append((key, item))

In [49]:
df_slug_email = pd.DataFrame(list_slug_email, columns=['slug', 'email'])

In [50]:
df_slug_email

,slug,email
0,spain-fs-pt-marzo,edgarbrignoni@icloud.com
1,spain-fs-pt-enero,alejandro.leon1223@gmail.com
2,miami-52,stallonedylan954@gmail.com
3,miami-52,a.a.lejo@gmail.com
4,miami-52,a+fakestudent8@4geeks.us
5,mdc-iii,imjosearce@gmail.com
6,miami-51,a+fakestudent8@4geeks.us
7,spain-fs-pt-mayo,mazda3mps265@hotmail.com
8,full-stack-11,calexandergarcia@gmail.com
9,full-stack-21,wambuangui0746@gmail.com


In [51]:
upcoming_cohorts_users = upcoming_cohorts.merge(df_slug_email, on='slug', how='left')

In [52]:
upcoming_cohorts_users

,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,updated_at,...,schedule_id,syllabus_version_id,online_meeting_url,remote_available,current_module,history_log,is_hidden_on_prework,campus,datetime_kickoff,email
0,501,spain-fs-pt-enero,Spain FS PT Enero,2023-06-06 14:16:00,2023-10-10 15:16:00,0,INACTIVE,es,2022-12-09 15:17:36.465700,2022-12-24 10:42:58.373921,...,128.0,182.0,NaN,False,NaN,NaN,NaN,4Geeks Academy Madrid,2023-06-06 14:16:00,alejandro.leon1223@gmail.com
1,503,spain-fs-pt-marzo,Spain FS PT Marzo,2023-03-07 15:23:00,2023-07-18 15:23:00,0,INACTIVE,es,2022-12-09 15:24:20.195068,2022-12-09 15:24:20.195078,...,128.0,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Madrid,2023-03-07 15:23:00,edgarbrignoni@icloud.com
2,380,fake-ml-cohort,Fake ML Cohort,2023-04-20 19:04:00,2023-12-19 20:04:00,0,STARTED,en,2022-06-15 19:04:29.787991,2023-02-02 13:40:50.675166,...,77.0,136.0,NaN,False,NaN,NaN,True,NaN,2023-04-20 19:04:00,a.alejo@gmail.com
3,473,latam-pt-20,latam-pt-20,2023-06-19 18:16:00,2023-10-20 18:16:00,0,INACTIVE,es,2022-10-18 18:17:28.070083,2023-01-16 22:41:35.109890,...,134.0,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Latam,2023-06-19 18:16:00,hexen6@gmail.com
4,470,latam-pt-17,latam-pt-17,2023-03-13 18:12:00,2023-07-17 18:12:00,0,INACTIVE,es,2022-10-18 18:14:09.717460,2023-01-16 22:42:17.213203,...,134.0,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Latam,2023-03-13 18:12:00,NaN
5,403,miami-50,Miami 50,2023-03-20 14:55:00,2023-07-21 14:55:00,0,PREWORK,en,2022-08-11 14:55:57.057573,2023-02-22 19:10:04.686012,...,1.0,61.0,https://google.com,True,NaN,NaN,False,4Geeks Academy Miami,2023-03-20 14:55:00,test@4geeksacademy.com
6,404,miami-51,Miami 51,2023-04-10 14:55:00,2023-08-11 14:55:00,0,INACTIVE,en,2022-08-11 14:56:35.541142,2023-02-13 14:19:28.670074,...,1.0,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-04-10 14:55:00,a+fakestudent8@4geeks.us
7,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,13.0,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-01 14:56:00,stallonedylan954@gmail.com
8,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,13.0,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-01 14:56:00,a.a.lejo@gmail.com
9,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,13.0,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-01 14:56:00,a+fakestudent8@4geeks.us


Add columns related to assigned

In [53]:
def check_in_dict(row, dict_x):
    if row['slug'] not in dict_x:
        return False
    else:
        return row['email'] in dict_x[row['slug']]

In [54]:
upcoming_cohorts_users['assigned'] = upcoming_cohorts_users.apply(lambda row: check_in_dict(row, dict_assigned), axis=1)

In [55]:
upcoming_cohorts_users

,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,updated_at,...,syllabus_version_id,online_meeting_url,remote_available,current_module,history_log,is_hidden_on_prework,campus,datetime_kickoff,email,assigned
0,501,spain-fs-pt-enero,Spain FS PT Enero,2023-06-06 14:16:00,2023-10-10 15:16:00,0,INACTIVE,es,2022-12-09 15:17:36.465700,2022-12-24 10:42:58.373921,...,182.0,NaN,False,NaN,NaN,NaN,4Geeks Academy Madrid,2023-06-06 14:16:00,alejandro.leon1223@gmail.com,False
1,503,spain-fs-pt-marzo,Spain FS PT Marzo,2023-03-07 15:23:00,2023-07-18 15:23:00,0,INACTIVE,es,2022-12-09 15:24:20.195068,2022-12-09 15:24:20.195078,...,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Madrid,2023-03-07 15:23:00,edgarbrignoni@icloud.com,False
2,380,fake-ml-cohort,Fake ML Cohort,2023-04-20 19:04:00,2023-12-19 20:04:00,0,STARTED,en,2022-06-15 19:04:29.787991,2023-02-02 13:40:50.675166,...,136.0,NaN,False,NaN,NaN,True,NaN,2023-04-20 19:04:00,a.alejo@gmail.com,True
3,473,latam-pt-20,latam-pt-20,2023-06-19 18:16:00,2023-10-20 18:16:00,0,INACTIVE,es,2022-10-18 18:17:28.070083,2023-01-16 22:41:35.109890,...,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Latam,2023-06-19 18:16:00,hexen6@gmail.com,False
4,470,latam-pt-17,latam-pt-17,2023-03-13 18:12:00,2023-07-17 18:12:00,0,INACTIVE,es,2022-10-18 18:14:09.717460,2023-01-16 22:42:17.213203,...,182.0,NaN,True,NaN,NaN,NaN,4Geeks Academy Latam,2023-03-13 18:12:00,NaN,False
5,403,miami-50,Miami 50,2023-03-20 14:55:00,2023-07-21 14:55:00,0,PREWORK,en,2022-08-11 14:55:57.057573,2023-02-22 19:10:04.686012,...,61.0,https://google.com,True,NaN,NaN,False,4Geeks Academy Miami,2023-03-20 14:55:00,test@4geeksacademy.com,True
6,404,miami-51,Miami 51,2023-04-10 14:55:00,2023-08-11 14:55:00,0,INACTIVE,en,2022-08-11 14:56:35.541142,2023-02-13 14:19:28.670074,...,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-04-10 14:55:00,a+fakestudent8@4geeks.us,False
7,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-01 14:56:00,stallonedylan954@gmail.com,True
8,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-01 14:56:00,a.a.lejo@gmail.com,True
9,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,61.0,NaN,True,NaN,NaN,False,4Geeks Academy Miami,2023-05-01 14:56:00,a+fakestudent8@4geeks.us,False


In [56]:
df_assigned = upcoming_cohorts_users[upcoming_cohorts_users['assigned']==True]

In [57]:
df_assigned = pd.merge(df_assigned, form_entry, left_on=['slug', 'email'], right_on=['ac_expected_cohort', 'email'])

In [58]:
df_assigned.columns

Index(['id_x', 'slug', 'name', 'kickoff_date', 'ending_date', 'current_day',
       'stage', 'language_x', 'created_at_x', 'updated_at_x', 'academy_id_x',
       'timezone', 'private', 'never_ends', 'schedule_id',
       'syllabus_version_id', 'online_meeting_url', 'remote_available',
       'current_module', 'history_log', 'is_hidden_on_prework', 'campus_x',
       'datetime_kickoff', 'email', 'assigned', 'id_y', 'first_name',
       'last_name', 'phone', 'course', 'client_comments', 'language_y',
       'utm_url', 'utm_medium', 'utm_campaign', 'street_address', 'country',
       'city', 'latitude', 'longitude', 'state', 'zip_code', 'storage_status',
       'created_at_y', 'updated_at_y', 'contact_id', 'lead_type', 'location',
       'referral_key', 'automations', 'tags', 'gclid', 'browser_lang',
       'fb_ad_id', 'fb_adgroup_id', 'fb_form_id', 'fb_leadgen_id',
       'fb_page_id', 'utm_source', 'academy_id_y', 'deal_status', 'sentiment',
       'ac_contact_id', 'ac_deal_id', 'user_i

In [59]:
grouped_assigned = df_assigned.groupby(['slug', 'email']).agg({'datetime_update': 'max', 'location': 'last', 'ac_deal_id':'last', 'ac_deal_owner_full_name':'last', 'created_at_y': 'last', 'campus_check':'last'})
grouped_assigned = grouped_assigned.reset_index()

In [60]:
upcoming_cohorts_users = upcoming_cohorts_users.merge(grouped_assigned, on='email', how='left').drop(columns=['slug_y', 'datetime_update'])

In [61]:
upcoming_cohorts_users = upcoming_cohorts_users.rename(columns={'slug_x':'slug', 'created_at_y':'form_created_at', 'location':'expected_location', 'ac_deal_id':'deal_id', 'ac_deal_owner_full_name':'deal_owner'})

In [62]:
upcoming_cohorts_users

,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,updated_at,...,is_hidden_on_prework,campus,datetime_kickoff,email,assigned,expected_location,deal_id,deal_owner,form_created_at,campus_check
0,501,spain-fs-pt-enero,Spain FS PT Enero,2023-06-06 14:16:00,2023-10-10 15:16:00,0,INACTIVE,es,2022-12-09 15:17:36.465700,2022-12-24 10:42:58.373921,...,NaN,4Geeks Academy Madrid,2023-06-06 14:16:00,alejandro.leon1223@gmail.com,False,NaN,NaN,NaN,NaN,NaN
1,503,spain-fs-pt-marzo,Spain FS PT Marzo,2023-03-07 15:23:00,2023-07-18 15:23:00,0,INACTIVE,es,2022-12-09 15:24:20.195068,2022-12-09 15:24:20.195078,...,NaN,4Geeks Academy Madrid,2023-03-07 15:23:00,edgarbrignoni@icloud.com,False,NaN,NaN,NaN,NaN,NaN
2,380,fake-ml-cohort,Fake ML Cohort,2023-04-20 19:04:00,2023-12-19 20:04:00,0,STARTED,en,2022-06-15 19:04:29.787991,2023-02-02 13:40:50.675166,...,True,NaN,2023-04-20 19:04:00,a.alejo@gmail.com,True,downtown-miami,NaN,None,2020-09-23 21:22:11.861392,Location does not match campus
3,473,latam-pt-20,latam-pt-20,2023-06-19 18:16:00,2023-10-20 18:16:00,0,INACTIVE,es,2022-10-18 18:17:28.070083,2023-01-16 22:41:35.109890,...,NaN,4Geeks Academy Latam,2023-06-19 18:16:00,hexen6@gmail.com,False,NaN,NaN,NaN,NaN,NaN
4,470,latam-pt-17,latam-pt-17,2023-03-13 18:12:00,2023-07-17 18:12:00,0,INACTIVE,es,2022-10-18 18:14:09.717460,2023-01-16 22:42:17.213203,...,NaN,4Geeks Academy Latam,2023-03-13 18:12:00,NaN,False,NaN,NaN,NaN,NaN,NaN
5,403,miami-50,Miami 50,2023-03-20 14:55:00,2023-07-21 14:55:00,0,PREWORK,en,2022-08-11 14:55:57.057573,2023-02-22 19:10:04.686012,...,False,4Geeks Academy Miami,2023-03-20 14:55:00,test@4geeksacademy.com,True,downtown-miami,83822.0,Alejandro Sanchez,2023-02-16 04:23:08.809197,Location matches campus
6,404,miami-51,Miami 51,2023-04-10 14:55:00,2023-08-11 14:55:00,0,INACTIVE,en,2022-08-11 14:56:35.541142,2023-02-13 14:19:28.670074,...,False,4Geeks Academy Miami,2023-04-10 14:55:00,a+fakestudent8@4geeks.us,False,NaN,NaN,NaN,NaN,NaN
7,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,False,4Geeks Academy Miami,2023-05-01 14:56:00,stallonedylan954@gmail.com,True,downtown-miami,17526.0,Amelia Pond,2020-09-23 21:22:11.861392,Location matches campus
8,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,False,4Geeks Academy Miami,2023-05-01 14:56:00,a.a.lejo@gmail.com,True,None,NaN,None,2020-09-23 21:22:11.861392,Location does not match campus
9,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,False,4Geeks Academy Miami,2023-05-01 14:56:00,a+fakestudent8@4geeks.us,False,NaN,NaN,NaN,NaN,NaN


Add columns related to confirmed

In [63]:
upcoming_cohorts_users['confirmed'] = upcoming_cohorts_users.apply(lambda row: check_in_dict(row, dict_confirmed), axis=1)

In [64]:
df_confirmed = upcoming_cohorts_users[upcoming_cohorts_users['confirmed']==True]

In [65]:
# df_confirmed = pd.merge(upcoming_cohorts_users, cohort_users_auth, left_on=['slug', 'email'], right_on=['slug', 'email'])
df_confirmed = pd.merge(df_confirmed, cohort_users_auth, left_on=['slug', 'email'], right_on=['slug', 'email'])

In [66]:
df_confirmed.columns

Index(['id', 'slug', 'name', 'kickoff_date', 'ending_date', 'current_day',
       'stage', 'language', 'created_at_x', 'updated_at_x', 'academy_id',
       'timezone', 'private', 'never_ends', 'schedule_id',
       'syllabus_version_id', 'online_meeting_url', 'remote_available',
       'current_module', 'history_log_x', 'is_hidden_on_prework', 'campus',
       'datetime_kickoff', 'email', 'assigned', 'expected_location', 'deal_id',
       'deal_owner', 'form_created_at', 'campus_check', 'confirmed', 'id_x',
       'role', 'finantial_status', 'educational_status', 'created_at_y',
       'updated_at_y', 'cohort_id', 'user_id', 'watching', 'history_log_y',
       'id_y', 'first_name', 'last_name', 'date_joined', 'datetime_update'],
      dtype='object')

In [67]:
grouped_confirmed = df_confirmed.groupby(['slug', 'email']).agg({'datetime_update': 'max', 'educational_status': 'last', 'created_at_y': 'last'})
grouped_confirmed = grouped_confirmed.reset_index()

In [68]:
upcoming_cohorts_users = upcoming_cohorts_users.merge(grouped_confirmed, on='email', how='left')

In [69]:
upcoming_cohorts_users = upcoming_cohorts_users.rename(columns={'slug_x':'slug', 'created_at_y':'inscription_created_at'})

In [70]:
upcoming_cohorts_users

,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,updated_at,...,expected_location,deal_id,deal_owner,form_created_at,campus_check,confirmed,slug_y,datetime_update,educational_status,inscription_created_at
0,501,spain-fs-pt-enero,Spain FS PT Enero,2023-06-06 14:16:00,2023-10-10 15:16:00,0,INACTIVE,es,2022-12-09 15:17:36.465700,2022-12-24 10:42:58.373921,...,NaN,NaN,NaN,NaN,NaN,True,spain-fs-pt-enero,2020-08-19 23:06:31.550416,GRADUATED,2020-08-19 23:06:31.550397+00:00
1,503,spain-fs-pt-marzo,Spain FS PT Marzo,2023-03-07 15:23:00,2023-07-18 15:23:00,0,INACTIVE,es,2022-12-09 15:24:20.195068,2022-12-09 15:24:20.195078,...,NaN,NaN,NaN,NaN,NaN,True,spain-fs-pt-marzo,2020-08-19 23:06:31.565371,GRADUATED,2020-08-19 23:06:31.565350+00:00
2,380,fake-ml-cohort,Fake ML Cohort,2023-04-20 19:04:00,2023-12-19 20:04:00,0,STARTED,en,2022-06-15 19:04:29.787991,2023-02-02 13:40:50.675166,...,downtown-miami,NaN,None,2020-09-23 21:22:11.861392,Location does not match campus,False,NaN,NaT,NaN,NaN
3,473,latam-pt-20,latam-pt-20,2023-06-19 18:16:00,2023-10-20 18:16:00,0,INACTIVE,es,2022-10-18 18:17:28.070083,2023-01-16 22:41:35.109890,...,NaN,NaN,NaN,NaN,NaN,True,latam-pt-20,2020-08-19 23:06:31.584140,GRADUATED,2020-08-19 23:06:31.584120+00:00
4,470,latam-pt-17,latam-pt-17,2023-03-13 18:12:00,2023-07-17 18:12:00,0,INACTIVE,es,2022-10-18 18:14:09.717460,2023-01-16 22:42:17.213203,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaT,NaN,NaN
5,403,miami-50,Miami 50,2023-03-20 14:55:00,2023-07-21 14:55:00,0,PREWORK,en,2022-08-11 14:55:57.057573,2023-02-22 19:10:04.686012,...,downtown-miami,83822.0,Alejandro Sanchez,2023-02-16 04:23:08.809197,Location matches campus,False,NaN,NaT,NaN,NaN
6,404,miami-51,Miami 51,2023-04-10 14:55:00,2023-08-11 14:55:00,0,INACTIVE,en,2022-08-11 14:56:35.541142,2023-02-13 14:19:28.670074,...,NaN,NaN,NaN,NaN,NaN,True,miami-51,2020-08-19 23:06:31.519746,SUSPENDED,2020-08-19 23:06:31.519727+00:00
7,404,miami-51,Miami 51,2023-04-10 14:55:00,2023-08-11 14:55:00,0,INACTIVE,en,2022-08-11 14:56:35.541142,2023-02-13 14:19:28.670074,...,NaN,NaN,NaN,NaN,NaN,True,miami-52,2020-08-19 23:06:31.532644,SUSPENDED,2020-08-19 23:06:31.532619+00:00
8,404,miami-51,Miami 51,2023-04-10 14:55:00,2023-08-11 14:55:00,0,INACTIVE,en,2022-08-11 14:56:35.541142,2023-02-13 14:19:28.670074,...,NaN,NaN,NaN,NaN,NaN,True,miami-53,2020-08-19 23:06:31.538457,SUSPENDED,2020-08-19 23:06:31.538438+00:00
9,405,miami-52,Miami 52,2023-05-01 14:56:00,2023-08-25 14:56:00,0,INACTIVE,en,2022-08-11 14:57:23.079281,2023-02-13 14:20:07.118536,...,downtown-miami,17526.0,Amelia Pond,2020-09-23 21:22:11.861392,Location matches campus,False,NaN,NaT,NaN,NaN


**Cleaning upcoming_cohorts**

In [71]:
upcoming_cohorts_users['days_until_kickoff'] = (upcoming_cohorts_users['datetime_kickoff'] - pd.Timestamp.now()).dt.days

In [72]:
upcoming_cohorts_users['form_created_at'] = upcoming_cohorts_users['form_created_at'].apply(correct_date)
upcoming_cohorts_users['inscription_created_at'] = upcoming_cohorts_users['inscription_created_at'].apply(correct_date)

In [73]:
upcoming_cohorts_users['form_created_at'] = pd.to_datetime(upcoming_cohorts_users['form_created_at'])
upcoming_cohorts_users['inscription_created_at'] = pd.to_datetime(upcoming_cohorts_users['inscription_created_at'])

In [74]:
upcoming_cohorts_users['days_since_form_creation'] = (pd.Timestamp.now()- upcoming_cohorts_users['form_created_at']).dt.days
upcoming_cohorts_users['days_since_inscription'] = (pd.Timestamp.now()- upcoming_cohorts_users['inscription_created_at']).dt.days

In [75]:
def create_link(row):
    if pd.isnull(row['deal_id']):
        return np.nan
    else:
        return 'https://4geeks.activehosted.com/app/deals/' + str(int(row['deal_id']))

In [76]:
upcoming_cohorts_users['ac_deal_link'] = upcoming_cohorts_users.apply(create_link, axis=1)

In [77]:
upcoming_cohorts_users['assigned'] = np.where(upcoming_cohorts_users['assigned'] == True, 1, 0)

In [78]:
upcoming_cohorts_users['confirmed'] = np.where(upcoming_cohorts_users['confirmed'] == True, 1, 0)

In [79]:
upcoming_cohorts_users['unconfirmed'] = upcoming_cohorts_users['assigned']

In [80]:
upcoming_cohorts_users.loc[upcoming_cohorts_users['confirmed'] == 1, 'unconfirmed'] = 0

In [81]:
upcoming_cohorts_users = upcoming_cohorts_users[['id', 'slug', 'name', 'stage', 'language', 'academy_id', 'campus', 'datetime_kickoff', 'days_until_kickoff', 'email', 'assigned', 'expected_location', 'ac_deal_link', 'deal_owner', 'form_created_at', 'days_since_form_creation', 'unconfirmed', 'confirmed', 'educational_status', 'inscription_created_at', 'days_since_inscription', 'campus_check']]

In [82]:
upcoming_cohorts_users = upcoming_cohorts_users.drop_duplicates(subset=['id', 'email'], keep='last')

In [83]:
upcoming_cohorts_users = upcoming_cohorts_users.sort_values(by=['days_until_kickoff'], ascending=True)

In [84]:
upcoming_cohorts_users

,id,slug,name,stage,language,academy_id,campus,datetime_kickoff,days_until_kickoff,email,...,ac_deal_link,deal_owner,form_created_at,days_since_form_creation,unconfirmed,confirmed,educational_status,inscription_created_at,days_since_inscription,campus_check
1,503,spain-fs-pt-marzo,Spain FS PT Marzo,INACTIVE,es,6,4Geeks Academy Madrid,2023-03-07 15:23:00,8,edgarbrignoni@icloud.com,...,NaN,NaN,NaT,NaN,0,1,GRADUATED,2020-08-19 23:06:31.565350,921.0,NaN
29,525,caracas-pt-42,Caracas PT 42,INACTIVE,es,2,4Geeks Academy Caracas,2023-03-13 15:26:00,14,josemfcheo@gmail.com,...,NaN,None,2020-09-25 02:34:58.527898,885.0,1,0,NaN,NaT,NaN,No campus assigned
4,470,latam-pt-17,latam-pt-17,INACTIVE,es,7,4Geeks Academy Latam,2023-03-13 18:12:00,14,NaN,...,NaN,NaN,NaT,NaN,0,0,NaN,NaT,NaN,NaN
5,403,miami-50,Miami 50,PREWORK,en,4,4Geeks Academy Miami,2023-03-20 14:55:00,21,test@4geeksacademy.com,...,https://4geeks.activehosted.com/app/deals/83822,Alejandro Sanchez,2023-02-16 04:23:08.809197,11.0,1,0,NaN,NaT,NaN,Location matches campus
28,510,santiago-pt-37,Santiago PT-37,INACTIVE,es,5,4Geeks Academy Santiago,2023-03-27 13:36:00,28,NaN,...,NaN,NaN,NaT,NaN,0,0,NaN,NaT,NaN,NaN
8,404,miami-51,Miami 51,INACTIVE,en,4,4Geeks Academy Miami,2023-04-10 14:55:00,42,a+fakestudent8@4geeks.us,...,NaN,NaN,NaT,NaN,0,1,SUSPENDED,2020-08-19 23:06:31.538438,921.0,NaN
17,504,spain-fs-pt-abril,Spain FS PT Abril,INACTIVE,es,6,4Geeks Academy Madrid,2023-04-10 15:24:00,42,NaN,...,NaN,NaN,NaT,NaN,0,0,NaN,NaT,NaN,NaN
18,471,latam-pt-18,latam-pt-18,INACTIVE,es,7,4Geeks Academy Latam,2023-04-17 18:14:00,49,NaN,...,NaN,NaN,NaT,NaN,0,0,NaN,NaT,NaN,NaN
2,380,fake-ml-cohort,Fake ML Cohort,STARTED,en,12,NaN,2023-04-20 19:04:00,52,a.alejo@gmail.com,...,NaN,None,2020-09-23 21:22:11.861392,886.0,1,0,NaN,NaT,NaN,Location does not match campus
27,527,latam-pt-ds-1,latam-pt-ds-1,INACTIVE,es,7,4Geeks Academy Latam,2023-04-24 17:40:00,56,NaN,...,NaN,NaN,NaT,NaN,0,0,NaN,NaT,NaN,NaN


In [89]:
upcoming_cohorts_users['days_since_inscription'].info()

<class 'pandas.core.series.Series'>
Int64Index: 24 entries, 1 to 20
Series name: days_since_inscription
Non-Null Count  Dtype  
--------------  -----  
6 non-null      float64
dtypes: float64(1)
memory usage: 384.0 bytes


In [86]:
upcoming_cohorts_users.to_csv('../output/upcoming_cohorts_users.csv', index=False)